# Experimento para detección semi-automática de recorridos, tomando agrupando por idsam + fecha

La idea es que utilizando el patrón dibujado por cada idsam dentro de un día se pueda detectar y agrupar los patrones de recorridos iguales/similares a manera de poder detectar la linea/ramal pudiendo ignorar el idrutaestacion.

Esto nos permitiria:
a) Auto-detectar el ramal por su trayectoria y poder empezar a limpiar el idrutaestacion 0000
b) Detectar alteraciones y anormalidades en el trayecto esperado.
c) Predecir congestión? Acá ya volando alto, pero el markdown perdona todo.

Importar la librerías que vamos a necesitar. Como vamos a verificar mucho del funcionamiento manualmente para poder desarrollar una metodologia, vamos a hacer varios mapas.

In [84]:
import pandas as pd
import numpy as np

import os
import datetime

import geopandas as gpd
import contextily as cx

import matplotlib.pyplot as plt

Carguemos los datos de una linea de prueba, para este caso vamos a usar la idrutaestacion __0145__ que esta asociada a __"ITA - LAMBARÉ (Canal 13) Troncal 1"__ de la empresa **"3 DE FEBRERO SA"**

In [85]:
datos_rutas = pd.read_csv("../../data/interim/datosporlinea/linea-0145.csv", sep = ';',low_memory=False)
print(datos_rutas.sample(10))

                            serialtarjeta           idsam   
236059   410c88f35af5af19ca9be8779183c9e3  0408211a846280  \
1001087  fbd8bac8ee88d25511a9bef5144d0505  041c1de24e6180   
977015   95ae078df088679373621215f27652d5  04081e1a846280   
373252   19af5021e4b21f41fe2092c579b55506  04191fe24e6180   
179737   e2ffdc5c0e4aa14c80c958dfdf783e1f  041917e24e6180   
461408   fd66b665e616c575eafc0d351d8692fc  04191be24e6180   
977018   25646f0392d70579d554dab3746e0f24  041d12e24e6180   
748397   a012ea48b721d3f3171408f48d9c1184  0408181a846280   
717496   130f9788241521aec97edd81ebfa6252  04303e22b95a80   
209662   23453c22abf9c675aeb9891687b45c78  0409131a846280   

                       fechahoraevento producto  montoevento   
236059   2022/11/18 13:28:46.000000000       MO         3400  \
1001087  2022/09/27 13:27:00.000000000       MO         2300   
977015   2022/08/06 07:00:30.000000000       MO         3400   
373252   2023/01/25 09:03:18.000000000       MO         2300   
179737  

Ahora empezamos a limpiar todos los datos que no nos interesan.

In [86]:
print(datos_rutas.columns.values.tolist())

['serialtarjeta', 'idsam', 'fechahoraevento', 'producto', 'montoevento', 'consecutivoevento', 'identidad', 'tipoevento', 'latitude', 'longitude', 'idrutaestacion', 'tipotransporte']


In [87]:
datos_rutas.drop(['serialtarjeta', 'producto', 'montoevento', 'consecutivoevento', 'identidad', 'tipoevento', 'tipotransporte'],inplace=True, axis=1)
print(datos_rutas.sample(10))

                 idsam                fechahoraevento  latitude  longitude   
259302  0408201a846280  2022/11/08 21:17:46.000000000 -25.34695  -57.50816  \
488882  0410221aaa6380  2023/02/02 05:14:14.000000000 -25.50021  -57.37599   
183512  04301222b95a80  2022/10/07 18:06:10.000000000 -25.32394  -57.55347   
436241  04301622b95a80  2023/02/23 14:04:04.000000000 -25.41678  -57.45645   
348405  0411171aaa6380  2022/12/04 08:12:58.000000000 -25.44156  -57.44257   
800564  041c21e24e6180  2022/06/11 20:26:30.000000000 -25.50241  -57.37337   
812177  04081e1a846280  2022/06/08 17:42:54.000000000 -25.31242  -57.58586   
291031  041c20e24e6180  2022/12/28 07:11:20.000000000 -25.45701  -57.36728   
243018  04301222b95a80  2022/11/16 06:53:40.000000000 -25.34860  -57.63507   
966621  04081b1a846280  2022/08/10 12:31:22.000000000 -25.50705  -57.35786   

        idrutaestacion  
259302             145  
488882             145  
183512             145  
436241             145  
348405          

Ahora que solamente tenemos los identificadores de los lectores (buses), los momentos en que ocurren los eventos, la posición geografica del evento (latitude,longitude) y el identificador de la ruta; ya podemos empezar a trabajar.

In [88]:
datos_rutas.info()
datos_rutas.sample(10)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1074281 entries, 0 to 1074280
Data columns (total 5 columns):
 #   Column           Non-Null Count    Dtype  
---  ------           --------------    -----  
 0   idsam            1074281 non-null  object 
 1   fechahoraevento  1074281 non-null  object 
 2   latitude         1074281 non-null  float64
 3   longitude        1074281 non-null  float64
 4   idrutaestacion   1074281 non-null  int64  
dtypes: float64(2), int64(1), object(2)
memory usage: 41.0+ MB


,idsam,fechahoraevento,latitude,longitude,idrutaestacion
139389,04301722b95a80,2022/10/24 07:56:48.000000000,-25.31706,-57.57223,145
343409,04191be24e6180,2022/12/06 17:07:46.000000000,-25.32289,-57.55542,145
219826,0408211a846280,2022/11/23 19:07:04.000000000,-25.43032,-57.44985,145
90372,041c24e24e6180,2022/02/13 22:24:38.000000000,-25.34455,-57.50161,145
1073078,0408171a846280,2022/09/01 14:35:00.000000000,-25.37601,-57.47669,145
478112,04191fe24e6180,2023/02/06 05:37:20.000000000,-25.37321,-57.47805,145
938826,041c21e24e6180,2022/08/21 18:17:08.000000000,-25.35558,-57.48846,145
115364,041c24e24e6180,2022/02/02 20:16:40.000000000,-25.31719,-57.57183,145
953769,04300e22b95a80,2022/08/15 08:50:28.000000000,-25.48000,-57.40960,145
878025,0411171aaa6380,2022/07/14 17:57:58.000000000,-25.31711,-57.57210,145


Primeramente revisamos los datos y hacemos algo de limpieza y ajustes.

In [93]:
# Convertimos la columna de cadena a fecha
datos_rutas['fechahoraevento']= pd.to_datetime(datos_rutas['fechahoraevento'])

# Remover todos los casos donde latitud y longitud sean ceros
datos_rutas = datos_rutas[(datos_rutas[['latitude','longitude']] != 0).all(axis=1)] 
# datos_rutas["fechastr"] = datos_rutas['fechahoraevento'].dt.strftime('%Y%m%d%H%M')
datos_rutas["fechastr"] = datos_rutas['fechahoraevento'].dt.strftime('%Y%m%d')

datos_rutas

,idsam,fechahoraevento,latitude,longitude,idrutaestacion,fechastr
0,041d12e24e6180,2022-01-30 23:49:56,-25.38953,-57.47013,145,20220130
1,041d12e24e6180,2022-01-30 23:40:46,-25.34554,-57.50541,145,20220130
2,041d12e24e6180,2022-01-30 23:40:40,-25.34554,-57.50541,145,20220130
3,041d12e24e6180,2022-01-30 23:40:34,-25.34554,-57.50541,145,20220130
4,041d12e24e6180,2022-01-30 23:40:30,-25.34554,-57.50541,145,20220130
...,...,...,...,...,...,...
1074276,04132822626c80,2022-09-01 05:22:52,-25.34199,-57.62491,145,20220901
1074277,04132822626c80,2022-09-01 05:22:22,-25.34224,-57.62633,145,20220901
1074278,04132822626c80,2022-09-01 05:20:38,-25.34265,-57.63326,145,20220901
1074279,04132822626c80,2022-09-01 05:20:38,-25.34265,-57.63326,145,20220901


Ahora vamos a tomar un idsam en particular y vamos a comenzar el trabajo de intentar crear una ruta estandar contra la cual vamos a comprar a las demas.

In [94]:
# Creamos un listado de todos los idsams (lectores-vehiculos) presentes en el dataset limpio
idsams = datos_rutas['idsam'].unique() 
print(idsams.shape)
print(idsams)
fechas_trayectorias = datos_rutas['fechastr'].unique() 
print(fechas_trayectorias.shape)
print(fechas_trayectorias)

(45,)
['041d12e24e6180' '04132822626c80' '04300e22b95a80' '0408191a846280'
 '04081b1a846280' '041922e24e6180' '04132f1aaa6380' '0408181a846280'
 '041c24e24e6180' '0408131a846280' '04191be24e6180' '041a38e24e6180'
 '041c21e24e6180' '04301222b95a80' '041c1ce24e6180' '041d14e24e6180'
 '041c20e24e6180' '04301522b95a80' '0408211a846280' '0413271aaa6380'
 '04081f1a846280' '0408171a846280' '0408201a846280' '04191fe24e6180'
 '041d0fe24e6180' '04081e1a846280' '04301722b95a80' '04082c1a846280'
 '0411171aaa6380' '04301622b95a80' '0408151a846280' '0409131a846280'
 '041917e24e6180' '041c1fe24e6180' '041c22e24e6180' '041c1de24e6180'
 '041918e24e6180' '04121b1aaa6380' '04131122626c80' '04301a22b95a80'
 '04303e22b95a80' '04120e1aaa6380' '0410221aaa6380' '04203ed2955d80'
 '04102e1aaa6380']
(440,)
['20220130' '20220129' '20220128' '20220127' '20220126' '20220125'
 '20220124' '20220123' '20220122' '20220121' '20220120' '20220119'
 '20220118' '20220117' '20220116' '20220115' '20220114' '20220113'
 '202201

In [91]:
# Tomar todos los datos para un idsam en particular
vehiculo_test = datos_rutas

# Asegurarnos que todos los valores estan ordenados por fecha
vehiculo_test = vehiculo_test.sort_values(by='fechahoraevento')

# Tomar solo muestras de un periodo en particular
dia = vehiculo_test[(vehiculo_test['fechahoraevento'] > "2021-12-31") & (vehiculo_test['fechahoraevento'] < "2023-01-01")]
minx, miny, maxx, maxy = gpd.GeoDataFrame(dia, geometry=gpd.points_from_xy(dia.longitude,dia.latitude),crs="EPSG:4326").total_bounds

print("Esquinas de la ruta para 0145: ",minx, miny, maxx, maxy)

-57.63565 -25.52047 -57.35287 -25.16842


Ahora tratemos de visualizar que paso por día con esos lectores

In [ ]:
fechasstr = datos_rutas['fechastr'].unique()
DIAS = ['Domingo','Lunes', 'Martes', 'Miércoles', 'Jueves', 'Viernes', 'Sábado']

for fechastr in fechasstr:
    imgpath = './graphs/'+fechastr+'.png'
    
    if(os.path.isfile(imgpath)):
        continue
    
    # print(imgpath)
    
    dia_test = dia[(dia[['fechastr']] == fechastr).all(axis=1)] 

    fecha_linda = fechastr[:4]+'-'+fechastr[4:6]+'-'+fechastr[6:8]
    fecha_linda = fecha_linda + " " + DIAS[int(datetime.datetime.strptime(fechastr, '%Y%m%d').strftime("%w"))]
    
    latlon_dia = gpd.GeoDataFrame(dia_test, geometry=gpd.points_from_xy(dia_test.longitude,dia_test.latitude),crs="EPSG:4326")
    try:
        mapa_dia_plot = latlon_dia.plot(figsize=(9, 9),legend=False,marker='+',markersize=20, column='idsam')
        mapa_dia_plot.set(title='Datos '+fecha_linda+' para el idrutaestacion 0145')
        cx.add_basemap(mapa_dia_plot, crs="EPSG:4326",source=cx.providers.Stamen.Toner)
        mapa_dia_plot.set_xlim(xmin=minx, xmax=maxx)
        mapa_dia_plot.set_ylim(ymin=miny, ymax=maxy)
        plt.axis('equal')
        plt.savefig(imgpath,dpi=300)
        plt.close()
    except:
        continue
 

# print("Graficos generados")

Ahora que tenemos cierta idea de cual es el moviento de cada lector, podemos comenzar a crear conjeturas de como podemos generar una trayectoria.
- Uno de los aprendizajes que nos dejan estas imagenes es que un lector no necesariamente esta unido a una sola ruta; un lector puede, durante el día o durante la semana, cambiar de ruta.
- Una misma linea cubre rutas diferentes; pero estas otras rutas son cubiertas con frecuencias diferentes. Casi puede notarse una trayectoria principal y otra secundaria(s); cuando el lector debe cubrir una ruta adicional.
- Las rectas que vamos a dibujar deberan ser por día y por idsam; y rectas que no se ajusten a lo normal deberan ser agrupadas aparte o descartadas. Esto puede generar posiblemente que un idrutaestacion genere como minimo una cantidad de rutas asignadas + 1


Como notas valiosas sobre las cuales vamos a trabajar ahora dejo dos links:
- [Exploring Trip Fuel Consumption by Machine Learning from GPS and CAN Bus Data](https://doi.org/10.11175/easts.11.906)
- [Nivel de exactitud (Accuracy) que agrega cada digito de un valor GPS](https://gis.stackexchange.com/questions/8650/measuring-accuracy-of-latitude-and-longitude)

In [132]:
# vehiculo_random = datos_rutas.sample(1).values[0]
vehiculo_random = ['041c24e24e6180', '2022-12-03 13:02:24', -25.31375,-57.58209, 145, '20221203']
vehiculo_test = datos_rutas[(datos_rutas[['idsam']] == vehiculo_random[0]).all(axis=1)] 
vehiculo_test = vehiculo_test[(vehiculo_test[['fechastr']] == vehiculo_random[5]).all(axis=1)] 
vehiculo_test = vehiculo_test.sort_values(by='fechahoraevento')
vehiculo_test

,idsam,fechahoraevento,latitude,longitude,idrutaestacion,fechastr
350580,041c24e24e6180,2022-12-03 12:12:50,-25.33292,-57.62089,145,20221203
350570,041c24e24e6180,2022-12-03 12:14:28,-25.33359,-57.62555,145,20221203
350543,041c24e24e6180,2022-12-03 12:21:42,-25.34205,-57.62656,145,20221203
350530,041c24e24e6180,2022-12-03 12:25:48,-25.35054,-57.63395,145,20221203
350523,041c24e24e6180,2022-12-03 12:27:32,-25.35188,-57.62821,145,20221203
...,...,...,...,...,...,...
348588,041c24e24e6180,2022-12-03 23:59:04,-25.33405,-57.53676,145,20221203
348587,041c24e24e6180,2022-12-03 23:59:20,-25.33429,-57.53614,145,20221203
348586,041c24e24e6180,2022-12-03 23:59:26,-25.33428,-57.53612,145,20221203
348585,041c24e24e6180,2022-12-03 23:59:36,-25.33445,-57.53567,145,20221203
